# Web Tree 구축

In [1]:
import os
import json
import time

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from webdriver_manager.chrome import ChromeDriverManager

In [2]:
def get_links_and_text(driver, url, root_url):
    """ 주어진 URL에서 내부 링크와 페이지의 텍스트 콘텐츠를 가져옴 """
    try:
        driver.get(url)
        time.sleep(2)  # 페이지 로딩 대기

        # driver.execute_script("window.localStorage.clear();")  # 로컬 스토리지 삭제
        # driver.execute_script("window.sessionStorage.clear();")  # 세션 스토리지 삭제
        # driver.delete_all_cookies()  # 쿠키 삭제

        # for i in range(2):
        #     driver.refresh()

        # 모든 내부 링크 가져오기
        links = set()
        elements = driver.find_elements(By.TAG_NAME, "a")
        for elem in elements:
            link = elem.get_attribute("href")
            if link and link.startswith(root_url):  # 내부 링크만 저장
                links.add(link)

        # 페이지의 텍스트 콘텐츠 가져오기
        page_content = driver.find_element(By.TAG_NAME, "body").text

        return links, page_content.strip()
    
    except Exception as e:
        print(f"Error while accessing {url}: {e}")
        return set(), ""

def crawl_website(root_name, root_url, max_depth=2, save_path="./"):
    """ 특정 사이트를 시작으로 내부 링크를 크롤링 (Depth별로 저장) 및 페이지 텍스트 저장 """
    MAX_NODES_PER_DEPTH = 10  # 각 depth에서 최대 크롤링할 노드 개수

    options = Options()
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--disable-gpu')
    user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36"
    options.add_argument(f"user-agent={user_agent}")    
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

    site_structure = {}  # Depth별 URL 저장
    page_contents = {}  # URL별 텍스트 저장
    visited = {}  # 방문한 URL 저장 (url: depth)
    depth_count = {}  # 각 depth에서 크롤링된 URL 개수 추적

    def crawl(url, depth):
        if depth > max_depth or url in visited:
            return

        # 루트(depth=0)는 하나만 허용, 나머지는 최대 10개까지 허용
        if depth > 0 and depth_count.get(depth, 0) >= MAX_NODES_PER_DEPTH:
            return

        visited[url] = depth
        depth_count[depth] = depth_count.get(depth, 0) + 1  # 해당 depth에서 크롤링된 노드 수 증가

        print(f"Crawling: {url} (Depth: {depth})")
        child_links, page_text = get_links_and_text(driver, url, root_url)

        # Depth별로 URL 저장
        if depth not in site_structure:
            site_structure[depth] = []
        site_structure[depth].append(url)

        # URL별 텍스트 저장
        page_contents[url] = page_text

        for link in child_links:
            crawl(link, depth + 1)

    # 루트 노드 크롤링 (오직 하나만)
    crawl(root_url, 0)
    driver.quit()

    # JSON 파일 저장 (사이트 구조)
    with open(f"{save_path}/{root_name}_site_structure.json", "w", encoding="utf-8") as f:
        json.dump(site_structure, f, indent=4, ensure_ascii=False)

    # JSON 파일 저장 (페이지 콘텐츠)
    with open(f"{save_path}/{root_name}_page_contents.json", "w", encoding="utf-8") as f:
        json.dump(page_contents, f, indent=4, ensure_ascii=False)

    return site_structure, page_contents

In [ ]:
root_name = "sap_korea"
root_url = "https://www.sap.com/korea"

save_path = "../data"
os.makedirs(save_path, exist_ok=True)
site_structure, page_contents = crawl_website(root_name, root_url, max_depth=15, save_path=save_path)

print(f"✅ {save_path}/{root_name}site_structure.json 저장 완료")
print(f"✅ {save_path}/{root_name}page_contents.json 저장 완료")

In [ ]:
# root_name = "현대자동차그룹"
# root_url = "https://audit.hyundai.com/"

# save_path = "../data"
# os.makedirs(save_path, exist_ok=True)
# site_structure, page_contents = crawl_website(root_name, root_url, max_depth=15, save_path=save_path)

# print(f"✅ {save_path}/{root_name}site_structure.json 저장 완료")
# print(f"✅ {save_path}/{root_name}page_contents.json 저장 완료")

In [ ]:
# root_name = "남양유업"
# root_url = "https://company.namyangi.com/"

# save_path = "../data"
# os.makedirs(save_path, exist_ok=True)
# site_structure, page_contents = crawl_website(root_name, root_url, max_depth=3, save_path=save_path)

# print(f"✅ {save_path}/{root_name}site_structure.json 저장 완료")
# print(f"✅ {save_path}/{root_name}page_contents.json 저장 완료")

# RAG

## Document

In [13]:
import os
import json
import glob

from uuid import uuid4
from langchain.docstore.document import Document

In [15]:
json_files = sorted(glob.glob("../data/*_page_contents.json"))
print(len(json_files))
print(json_files)

3
['../data/sap_korea_page_contents.json', '../data/남양유업_page_contents.json', '../data/현대자동차그룹_page_contents.json']


In [21]:
def load_documents(file_list):
    total_docs = []
    for json_path in file_list:
        company_name = json_path.split('/')[-1].split('_')[0]
        with open(json_path, "r", encoding="utf-8") as f:
            page_contents = json.load(f)
            print(f"{company_name} - {len(page_contents)}")

        documents = [
            Document(
                page_content=text,
                metadata={"company_name" : company_name, "url": url},
                id=uuid4()
            )
            for url, text in page_contents.items()
        ]
        total_docs.extend(documents)

    return total_docs

In [22]:
documents = load_documents(json_files)
print(len(documents))

sap - 151
남양유업 - 151
현대자동차그룹 - 22
324


## Vector DB

Reference
- [https://wikidocs.net/234016](https://wikidocs.net/234016)

In [23]:
import os
import faiss

from datetime import datetime

from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore

from langchain_openai import OpenAIEmbeddings
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

from dotenv import load_dotenv
load_dotenv("../keys.env")
OPENAI_API_KEY = os.getenv("GRAVY_LAB_OPENAI")

In [24]:
def load_embed_model(provider, model_name, api_key=None, model_kwargs=None, encode_kwargs=None):
    if provider == "openai":
        model = OpenAIEmbeddings(model=model_name, api_key=api_key)
    elif provider == "huggingface":
        model = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs)

    return model

def create_vector_db(index_type, embed_model):
    sample_query = "Hello_World!"

    if index_type == "L2": ## L2 Distance(Euclidean Distance)
        index = faiss.IndexFlatL2(len(embed_model.embed_query(sample_query)))

    elif index_type == "IP": ## Inner Product
        index = faiss.IndexFlatIP(len(embed_model.embed_query(sample_query)))

    elif index_type == "HNSW":## ANN -> Hierarchical Navigable Small World
        index = faiss.IndexHNSWFlat(len(embed_model.embed_query(sample_query)))

    vector_db = FAISS(
        embedding_function=embed_model,
        index=index,
        docstore=InMemoryDocstore(),
        index_to_docstore_id={},
    )

    return vector_db

In [25]:
def save_vector_db(vector_db, save_path):
    vector_db.save_local(save_path)

def load_vector_db(index_file_path, embed_model):
    vector_db = FAISS.load_local(index_file_path, embeddings=embed_model, allow_dangerous_deserialization=True)

    return vector_db

In [26]:
now = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
save_path = f"../indexes/{now}"
os.makedirs(save_path, exist_ok=True)

In [27]:
provider = "huggingface"
model_name = "intfloat/multilingual-e5-large-instruct"
model_kwargs = {"device" : "cuda"}
encode_kwargs = {"normalize_embeddings": True}
index_type = "IP" ## "L2", "IP", "HNSW"

embed_model = load_embed_model(provider,
                               model_name,
                               api_key=OPENAI_API_KEY,
                               model_kwargs=model_kwargs, 
                               encode_kwargs=encode_kwargs)

vector_db = create_vector_db(index_type, embed_model)
vector_db.add_documents(documents)
vector_db.save_local(save_path)

In [12]:
# retriever = vector_db.as_retriever()

In [ ]:
# results = retriever.invoke("기업의 비전에 대해 알려주세요.",
#                            search_type="mmr",
#                            search_kwargs={"k" : 5, "fetch_k" : 10, "lambda_mult" : 0.5})

# for idx, result in enumerate(results):
#     print("=" * 100)
#     print(f"Result {idx:>04}")
#     print(f"{result.metadata['url']}\n{result.page_content}\n\n")

In [ ]:
# results = retriever.invoke("기업의 비전에 대해 알려주세요.",
#                            search_type="similarity_score_threshold",
#                            search_kwargs={"score_threshold" : 0.55})

# for idx, result in enumerate(results):
#     print("=" * 100)
#     print(f"Result {idx:>04}")
#     print(f"{result.metadata['url']}\n{result.page_content}\n\n")

In [28]:
results = vector_db.similarity_search_with_relevance_scores("현대자동차그룹의 비전에 대해 알려주세요.", k=10)

for document, score in results:
    print("=" * 100)
    print(f"{document.id}, {document.metadata['url']}")
    print(f"{score:.4f}")
    print(f"{document.page_content}\n\n")

6c3cfa1d-bf09-43c7-bdd1-2e68b1fe579e, https://audit.hyundai.com/progress
0.3755
기업이념
윤리규범
제보하기
제보하기
추진경과
현대자동차그룹은 국민행복과 국가발전을 위해
‘모범적인 기업’으로서의 역할에 더욱 책임을 다하겠습니다.
기업이념
추진경과
추진경과
현대자동차그룹은 사회적 책임을 다하기 위해 윤리경영을 적극 실천하고 있습니다.
2023
2022
2021
2020
2019
2018
기아, 온실가스 감축 위한 ‘다자 협력’ 나선다
현대자동차, 서울시에 시각장애인 맞춤 복지차량 기증
현대모비스, 10년간 조성한 생태숲에서 멸종위기종 복원
현대자동차그룹, 임직원과 함께하는 기부 캠페인 ‘기부해봄’ 실시
현대글로비스, 지구의 날 맞아 서울숲 정화활동 플로깅 실시
현대건설, ‘CDP 코리아 어워드’ 기후변화 부문 2년 연속 최상위 등급 선정
사이트 맵
개인정보처리방침
한국 웹접근성 인증
서울특별시 서초구 헌릉로 12(양재동)
02-3464-3500
02-3464-8813
KOR
COPYRIGHT ⓒ HYUNDAI MOTOR GROUP. ALL RIGHTS RESERVED.


007c2e11-9821-44fe-9e28-4bbf7d9c2c72, https://audit.hyundai.com/philosophy/23#contentPhilosophy
0.3784
기업이념
윤리규범
제보하기
제보하기
기업이념체계
현대자동차그룹은 국민행복과 국가발전을 위해
‘모범적인 기업’으로서의 역할에 더욱 책임을 다하겠습니다.
기업이념
기업이념체계
현대자동차그룹은 창립 이래 우리만의 고유한 정신과 가치를 계승하고 실천함으로써 지속 성장을 이루어 왔습니다. 이렇듯 현대자동차그룹이 추구하고 지켜나가야 할 가치와 변화의 방향, 그리고 미래상을 유기적으로 구성한 ‘경영이념체계’는 또 하나의 역사를 만들어 가는 구성원 모두의 목표이자 구심점입니다. 경영철학, 비전, 핵심가치를 이해관계자들과 공유함으로써, 구체적인 실천을 약속합니다.
경영

In [29]:
results = vector_db.similarity_search_with_relevance_scores("현대자동차그룹의 경영철학에 대해 알려주세요.", k=10)

for document, score in results:
    print("=" * 100)
    print(f"{document.id}, {document.metadata['url']}")
    print(f"{score:.4f}")
    print(f"{document.page_content}\n\n")

39a602c9-0ea7-4aaf-b391-6e302bbe49f6, https://audit.hyundai.com/philosophy/22#contentPhilosophy
0.3583
기업이념
윤리규범
제보하기
제보하기
기업이념체계
현대자동차그룹은 국민행복과 국가발전을 위해
‘모범적인 기업’으로서의 역할에 더욱 책임을 다하겠습니다.
기업이념
기업이념체계
현대자동차그룹은 창립 이래 우리만의 고유한 정신과 가치를 계승하고 실천함으로써 지속 성장을 이루어 왔습니다. 이렇듯 현대자동차그룹이 추구하고 지켜나가야 할 가치와 변화의 방향, 그리고 미래상을 유기적으로 구성한 ‘경영이념체계’는 또 하나의 역사를 만들어 가는 구성원 모두의 목표이자 구심점입니다. 경영철학, 비전, 핵심가치를 이해관계자들과 공유함으로써, 구체적인 실천을 약속합니다.
경영철학
핵심가치
비전
핵심가치
도전과 열정으로 내재된 핵심가치를 실천할 때
현대자동차그룹의 심장은 더욱 힘차게 뜁니다.
현대자동차그룹의 경영철학은 전 임직원들의 가슴 깊이 새겨져 그룹의 토대를 다지는 굳건한 정신입니다. 아무리 훌륭한 이론도 실천되지 않는다면 가치를 발휘할 수 없습니다. 현대자동차그룹의 핵심가치는 경영철학의 현장 실천을 구체적으로 대변하고 있습니다. 언제나 고객을 최우선으로 생각하고 도전적 실행을 통해 가능성을 실현하며, 소통과 협력의 조직 문화를 구축하고 개개인의 능력을 최대한 발현시키는 인재 존중의 정신으로 글로벌 역량을 키워나갈 것입니다.
Customer
고객최우선
최고의 품질과 최상의 서비스를 제공함으로써 모든 가치의 중심에 고객을 최우선으로 두는 고객감동의 기업 문화를 조성합니다.
Challenge
도전적 실행
현실에 안주하지 않고 새로운가능성에 도전하며 ‘할 수 있다’는 열정과 창의적 사고로 반드시 목표를 달성합니다.
Collaboration
소통과 협력
타 부문 및 협력사에 대한 상호 소통과 협력을 통해 ‘우리’라는 공동체 의식을 나눔으로써 시너지효과를 창출합니다.
People
인재존중
우리 조직의 미래가

In [32]:
results = vector_db.similarity_search_with_score("현대자동차그룹의 비전에 대해 알려주세요.", k=10)

for document, score in results:
    print("=" * 100)
    print(f"{document.id}, {document.metadata['url']}")
    print(f"Score : {score:.4f}")
    print(f"{document.page_content}\n\n")

6c3cfa1d-bf09-43c7-bdd1-2e68b1fe579e, https://audit.hyundai.com/progress
Score : 0.8832
기업이념
윤리규범
제보하기
제보하기
추진경과
현대자동차그룹은 국민행복과 국가발전을 위해
‘모범적인 기업’으로서의 역할에 더욱 책임을 다하겠습니다.
기업이념
추진경과
추진경과
현대자동차그룹은 사회적 책임을 다하기 위해 윤리경영을 적극 실천하고 있습니다.
2023
2022
2021
2020
2019
2018
기아, 온실가스 감축 위한 ‘다자 협력’ 나선다
현대자동차, 서울시에 시각장애인 맞춤 복지차량 기증
현대모비스, 10년간 조성한 생태숲에서 멸종위기종 복원
현대자동차그룹, 임직원과 함께하는 기부 캠페인 ‘기부해봄’ 실시
현대글로비스, 지구의 날 맞아 서울숲 정화활동 플로깅 실시
현대건설, ‘CDP 코리아 어워드’ 기후변화 부문 2년 연속 최상위 등급 선정
사이트 맵
개인정보처리방침
한국 웹접근성 인증
서울특별시 서초구 헌릉로 12(양재동)
02-3464-3500
02-3464-8813
KOR
COPYRIGHT ⓒ HYUNDAI MOTOR GROUP. ALL RIGHTS RESERVED.


007c2e11-9821-44fe-9e28-4bbf7d9c2c72, https://audit.hyundai.com/philosophy/23#contentPhilosophy
Score : 0.8791
기업이념
윤리규범
제보하기
제보하기
기업이념체계
현대자동차그룹은 국민행복과 국가발전을 위해
‘모범적인 기업’으로서의 역할에 더욱 책임을 다하겠습니다.
기업이념
기업이념체계
현대자동차그룹은 창립 이래 우리만의 고유한 정신과 가치를 계승하고 실천함으로써 지속 성장을 이루어 왔습니다. 이렇듯 현대자동차그룹이 추구하고 지켜나가야 할 가치와 변화의 방향, 그리고 미래상을 유기적으로 구성한 ‘경영이념체계’는 또 하나의 역사를 만들어 가는 구성원 모두의 목표이자 구심점입니다. 경영철학, 비전, 핵심가치를 이해관계자들과 공유함으로써, 구체